In [6]:
from patchEmbedding import patchEmbedding
from omegaconf import OmegaConf

conf = OmegaConf.load("configs/model.yml")
conf

{'loader': {'batch_size': 1, 'num_workers': 4}, 'patchEmbedding': {'in_channels': 3, 'out_channels': 384, 'patch_size': 16}, 'classification': {'hidden_dim': 384, 'proj_dim': 1000, 'num_class': 10}, 'backbone': {'hidden_dim': 384, 'channel_hidden_dim': 1024, 'index': 7, 'kernel_size': [1, 3, 5, 7], 'dilation': [1, 1, 1, 1], 'padding': [0, 1, 2, 3], 'num_mixers': 4}, 'optimizer': {'lr': 0.0005, 'betas': [0.9, 0.999], 'eps': 1e-08}}

In [7]:
patch_embedding = patchEmbedding(conf["patchEmbedding"])
patch_embedding

patchEmbedding(
  (patch_embedding): Sequential(
    (0): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    (1): Rearrange('b c h w -> b (h w) c')
  )
)

In [8]:
import torch
print(torch.__version__)

img = torch.ones([1, 3, 32, 32])
# img
outs = patch_embedding(img)
outs.shape

1.8.1+cu102


torch.Size([1, 4, 384])

In [9]:
from classifier import ImageCls

cls = ImageCls(conf["classification"]).cuda()
print(cls)
outs = cls(outs.cuda())
outs.shape

ImageCls(
  (feature_proj): Linear(in_features=384, out_features=1000, bias=True)
  (attention_proj): Linear(in_features=384, out_features=1000, bias=True)
  (cls_proj): Linear(in_features=1000, out_features=10, bias=True)
)


RuntimeError: CUDA error: no kernel image is available for execution on the device

In [68]:
import torch
import numpy as np
img = np.arange(0, 3*3*3).reshape(1, 3, 3, 3)
img

array([[[[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8]],

        [[ 9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]],

        [[18, 19, 20],
         [21, 22, 23],
         [24, 25, 26]]]])

In [74]:
import torch
import numpy as np
img = np.arange(0, 3*4*4).reshape(1, 3, 4, 4)
img = torch.LongTensor(img)
img


tensor([[[[ 0,  1,  2,  3],
          [ 4,  5,  6,  7],
          [ 8,  9, 10, 11],
          [12, 13, 14, 15]],

         [[16, 17, 18, 19],
          [20, 21, 22, 23],
          [24, 25, 26, 27],
          [28, 29, 30, 31]],

         [[32, 33, 34, 35],
          [36, 37, 38, 39],
          [40, 41, 42, 43],
          [44, 45, 46, 47]]]])

In [75]:
img = torch.roll(img, (1, 1, 1), (1, 2, 3))
img

tensor([[[[47, 44, 45, 46],
          [35, 32, 33, 34],
          [39, 36, 37, 38],
          [43, 40, 41, 42]],

         [[15, 12, 13, 14],
          [ 3,  0,  1,  2],
          [ 7,  4,  5,  6],
          [11,  8,  9, 10]],

         [[31, 28, 29, 30],
          [19, 16, 17, 18],
          [23, 20, 21, 22],
          [27, 24, 25, 26]]]])

In [73]:
img = torch.roll(img, (1), (2))
img

tensor([[[[44, 45, 46, 47],
          [32, 33, 34, 35],
          [36, 37, 38, 39],
          [40, 41, 42, 43]],

         [[12, 13, 14, 15],
          [ 0,  1,  2,  3],
          [ 4,  5,  6,  7],
          [ 8,  9, 10, 11]],

         [[28, 29, 30, 31],
          [16, 17, 18, 19],
          [20, 21, 22, 23],
          [24, 25, 26, 27]]]])

In [ ]:
img = torch.